In [33]:
import pandas as pd
import numpy as np
import sys
import os
import pyautogui
from bs4 import BeautifulSoup
from IPython.display import display, HTML
from datetime import datetime
import re
import time
import datetime
import cv2

from praisebot import move_and_click, group_locations
import praisebot

# import footballmanager
from footballmanager import parse_html_table, open_dir, convert_to_numeric
from footballmanager import imperial_to_decimal_height, weight_to_decimal_weight, convert_percent_string_to_decimal
from footballmanager import determine_goal_differential, determine_expires_date


import importlib

In [3]:
importlib.reload(praisebot)

<module 'praisebot' from 'C:\\Users\\lover\\OneDrive\\Documents\\GitHub\\football_manager_code\\AI\\praisebot.py'>

In [23]:
VIEW_DROPDOWN = (378, 259)
CUSTOM_MENU = (378, 319)
LEFT_PLAYER_CHECK = (137, 564)
TEAM_NAME = 'Atlético_Junior'

In [5]:
praisebot.navigate_to_fm_window()
pyautogui.hotkey('F3')
#time.sleep(2)
move_and_click(VIEW_DROPDOWN, 2)
move_and_click(CUSTOM_MENU, 2)

# Define the region of the screen to capture (x, y, width, height)
views_region = (692, 128, 828, 2012)


# Take a screenshot of the expanded menu
screenshot = pyautogui.screenshot(region=views_region)
screenshot = np.array(screenshot)
screenshot = cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

# Load the reference image and ensure it is in the correct format
reference_image_path = 'squad_attrib_selection.png'
reference_image = cv2.imread(reference_image_path, cv2.IMREAD_COLOR)

if reference_image is None:
    raise ValueError("Reference image not found or unable to load.")

# Find the reference image within the screenshot
result = cv2.matchTemplate(screenshot, reference_image, cv2.TM_CCOEFF_NORMED)

# Define a threshold for matching
threshold = 0.80
locations = np.where(result >= threshold)
locations = list(zip(*locations[::-1]))

# Group the locations
grouped_locations = group_locations(locations)

# Process each matching location
for loc in grouped_locations:
    center_x = loc[0] + reference_image.shape[1] // 2
    center_y = loc[1] + reference_image.shape[0] // 2
    screen_x = views_region[0] + center_x
    screen_y = views_region[1] + center_y

    pyautogui.click(screen_x, screen_y)
    time.sleep(0.5)  # Retain this sleep for precision timing
    print("Print squad view and breaking")
    break # Only click once for multiple views in the region

    time.sleep(0.2)  # Add a small delay between actions



Football Manager 2024 window is now active.
Print squad view and breaking


In [7]:
def click_on_located_image(region, image_name):
   # Define the region of the screen to capture (x, y, width, height)
   # views_region = (692, 128, 828, 2012)

    # Take a screenshot of the expanded menu
    screenshot = pyautogui.screenshot(region=region)
    screenshot = np.array(screenshot)
    screenshot = cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

    print("Screenshot taken")
          
    # Load the reference image and ensure it is in the correct format
    reference_image_path = image_name
    reference_image = cv2.imread(reference_image_path, cv2.IMREAD_COLOR)

    print("Reference image loaded")
          
    if reference_image is None:
        raise ValueError("Reference image not found or unable to load.")

        
    
    # Find the reference image within the screenshot
    result = cv2.matchTemplate(screenshot, reference_image, cv2.TM_CCOEFF_NORMED)

    print("Reference image found")
    
    # Define a threshold for matching
    threshold = 0.80
    locations = np.where(result >= threshold)
    locations = list(zip(*locations[::-1]))

    print (f"Length of locations {len(locations)}")
    
    # Group the locations
    grouped_locations = group_locations(locations)

    print(f"Number of grouped locations {len(grouped_locations)}")
    
    # Process each matching location
    for loc in grouped_locations:
        center_x = loc[0] + reference_image.shape[1] // 2
        center_y = loc[1] + reference_image.shape[0] // 2
        screen_x = region[0] + center_x
        screen_y = region[1] + center_y

        pyautogui.click(screen_x, screen_y)
        print(f"Clicked on {reference_image}")
        time.sleep(0.5)  # Retain this sleep for precision timing
        print(f"Image {image_name} located and clicked on")
        break # Only click once for multiple views in the region

        time.sleep(0.2)  # Add a small delay between actions

In [24]:
def gen_timestamp_filename():
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d_%H_%M_%S_") + TEAM_NAME

In [25]:
gen_timestamp_filename()

'2025-06-02_20_51_16_Atlético_Junior'

In [29]:
def open_squad_view():
    move_and_click(VIEW_DROPDOWN, 2)
    move_and_click(CUSTOM_MENU, 2)

    # Define the region of the screen to capture (x, y, width, height)
    views_region = (692, 128, 828, 2012)
    squad_attrib_image = 'images//squad_attrib_selection.png'
    click_on_located_image(views_region, squad_attrib_image)

        
def export_squad_data():
    # Move to top left player check
    move_and_click(LEFT_PLAYER_CHECK, 2)

    # Select all players
    pyautogui.hotkey('ctrl', 'a')
    time.sleep(2)
    
    # Print dialogue
    pyautogui.hotkey('ctrl', 'p')
    time.sleep(5)
    # Locate and click the OK button
    
    # Define the region of the screen to capture (x, y, width, height)
    print_dialogue_region = (0, 0, 3839,2159)
    ok_button_path = 'images//printer_ok_button.png'
    print(f"Attempting to click {ok_button_path}")
    click_on_located_image(print_dialogue_region, ok_button_path)
    print(f"{ok_button_path} clicked")
    
    # We need to find the circular "X" button, then navigate some pixels to the
    # 2322, 715
    output_file_region = (2240, 640, 82, 75)
    round_x_circle_path = 'images//round_x_circle.png'
    print(f"Attempting to click {round_x_circle_path}")
    click_on_located_image(output_file_region, round_x_circle_path)
    print(f"{round_x_circle_path} clicked")
    
    # Move cursor some pixels to the left
    pyautogui.move(-85, 0)
    # Click in form box 
    pyautogui.click()
    # Name the file by timestamp
    export_data_filename = gen_timestamp_filename()
    # Enter file name
    pyautogui.write(export_data_filename)
    time.sleep(2)
    pyautogui.press('enter')
    time.sleep(1)

In [32]:
if __name__ == "__main__":
    praisebot.navigate_to_fm_window()
    pyautogui.hotkey('F3')
    open_squad_view()
    export_squad_data()

Failed to activate Football Manager 2024 window: There are 2 elements that match the criteria {'title_re': 'Football Manager 2024', 'backend': 'win32', 'visible_only': False}


NameError: name 'sys' is not defined